In [1]:
import json
import pymysql
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, ConceptsOptions, CategoriesOptions, EmotionOptions, EntitiesOptions, KeywordsOptions, MetadataOptions, RelationsOptions, SemanticRolesOptions, SentimentOptions

In [2]:
natural_language_understanding = NaturalLanguageUnderstandingV1(url="https://gateway-fra.watsonplatform.net/natural-language-understanding/api",
                                                                username='545acdbc-0fa8-4807-ac6a-be86d3ca158c',
                                                                password='2SewOkZVnxrq',
                                                                version='2018-03-16')

In [3]:
def askWatson(text):
    response = natural_language_understanding.analyze(text=text,
                                                      return_analyzed_text=False,
                                                      features=Features(
                                                          #concepts=ConceptsOptions(),
                                                          #categories=CategoriesOptions(),
                                                          emotion=EmotionOptions(),
                                                          entities=EntitiesOptions(emotion=True,
                                                                                   limit=5),
                                                          #keywords=KeywordsOptions(),
                                                          #relations=RelationsOptions(),
                                                          #semantic_roles=SemanticRolesOptions(),
                                                          sentiment=SentimentOptions()))

    return response

In [6]:
try:
    conn = pymysql.connect(host='192.168.0.11', port=3306, user='root', passwd='geheim', db='mysql')
    
    cur = conn.cursor()
    cur.execute('USE twistory')

    cur.execute('SHOW COLUMNS FROM tweets')

    for ans in cur:
        print(ans)
    
except:
    raise
    
finally:
    cur.close()
    conn.close()

('tweet_id', 'bigint(20) unsigned', 'NO', 'PRI', None, '')
('user_id', 'bigint(20) unsigned', 'NO', 'MUL', None, '')
('tweet_text', 'varchar(400)', 'NO', 'MUL', None, '')
('name', 'varchar(20)', 'YES', 'MUL', None, '')
('created_at', 'datetime', 'NO', 'MUL', None, '')
('geo_lat', 'decimal(10,5)', 'YES', '', None, '')
('geo_long', 'decimal(10,5)', 'YES', '', None, '')
('screen_name', 'char(20)', 'NO', 'MUL', None, '')
('lang', 'varchar(50)', 'NO', '', None, '')
('is_rt', 'tinyint(1)', 'NO', '', None, '')
('sentiment', 'float', 'YES', '', None, '')
('sadness', 'float', 'YES', '', None, '')
('joy', 'float', 'YES', '', None, '')
('fear', 'float', 'YES', '', None, '')
('disgust', 'float', 'YES', '', None, '')
('anger', 'float', 'YES', '', None, '')


In [29]:
try:
    conn = pymysql.connect(host='192.168.0.11', port=3306, user='root', passwd='geheim', db='mysql')
    
    cur = conn.cursor()
    cur.execute('USE twistory')

    cur.execute('SELECT tweet_id, tweet_text FROM tweets WHERE watson_processed=0 LIMIT 5')

    data = cur.fetchall()
    for tweet_id, tweet_text in data:
        try:
            response = askWatson(tweet_text)
            statement = 'UPDATE tweets SET sentiment=%s, sadness=%s, joy=%s, fear=%s, disgust=%s, anger=%s, watson_processed=%s WHERE tweet_id=%s'
            
            cur.execute(statement, [response['sentiment']['document']['score'], response['emotion']['document']['emotion']['sadness'], response['emotion']['document']['emotion']['joy'], response['emotion']['document']['emotion']['fear'], response['emotion']['document']['emotion']['disgust'], response['emotion']['document']['emotion']['anger'], 1, tweet_id])
            conn.commit()
        except:
            statement = 'UPDATE tweets SET watson_processed=%s WHERE tweet_id=%s'
            cur.execute(statement, [1, tweet_id])
            conn.commit()
    
    
    
except:
    raise
    
finally:
    cur.close()
    conn.close()

No
Yes
Commit


In [30]:
try:
    conn = pymysql.connect(host='192.168.0.11', port=3306, user='root', passwd='geheim', db='mysql')
    
    cur = conn.cursor()
    cur.execute('USE twistory')

    cur.execute('SELECT * FROM tweets LIMIT 5')

    data = cur.fetchall()
    for row in data:
        print(row)
        
    
    
    
except:
    raise
    
finally:
    cur.close()
    conn.close()

(1005414395562864641, 4156223117, 'RT @Linus2k: Io non rosico.\n\nIo mi vergogno.\n\n#conte #g7 #canada', None, 'Sat Jun 09 11:40:46 +0000 2018', None, None, 'zan514', 'it', 1, None, None, None, None, None, None, 1)
(1005414405989945347, 588529368, 'RT @RedTRaccoon: Trump explaining in the most basic terms why Russia should be part of the #G72018 summit.\n\nOur President has flipped sides\x85', None, 'Sat Jun 09 11:40:48 +0000 2018', None, None, 'JgoBluewave', 'en', 1, 0.569252, 0.409083, 0.105143, 0.050645, 0.439003, 0.207611, 1)


In [13]:
response

{'emotion': {'document': {'emotion': {'anger': 0.207611,
    'disgust': 0.439003,
    'fear': 0.050645,
    'joy': 0.105143,
    'sadness': 0.409083}}},
 'entities': [{'count': 1,
   'disambiguation': {'subtype': ['Country']},
   'emotion': {'anger': 0.223246,
    'disgust': 0.065413,
    'fear': 0.064771,
    'joy': 0.130513,
    'sadness': 0.315629},
   'relevance': 0.868667,
   'text': 'Russia',
   'type': 'Location'},
  {'count': 1,
   'emotion': {'anger': 0.31644,
    'disgust': 0.369347,
    'fear': 0.080315,
    'joy': 0.092551,
    'sadness': 0.311468},
   'relevance': 0.814149,
   'text': 'President',
   'type': 'JobTitle'},
  {'count': 1,
   'relevance': 0.814149,
   'text': '@RedTRaccoon',
   'type': 'TwitterHandle'},
  {'count': 1, 'relevance': 0.814149, 'text': '#G72018', 'type': 'Hashtag'}],
 'language': 'en',
 'sentiment': {'document': {'label': 'positive', 'score': 0.569252}},
 'usage': {'features': 3, 'text_characters': 140, 'text_units': 1}}